In [13]:
import tensorflow as tf
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Conv2DTranspose
import numpy as np
import cupy as np_
import keras
import csv
#input (batch, rows, cols, channels)
#output (batch, new_rows, new_cols, filters)
#kernel (height,width,next_che,previ_che)
tf.random.set_seed(1)
inputs = tf.convert_to_tensor(np.arange(24).reshape((1,4,3,2)).astype(np.float32))
x=tf.keras.layers.Conv2DTranspose(filters = 7, kernel_size = (5,6), strides = (1,1), padding = 'valid')
y=x(inputs)
"""
print("input")
print(inputs)
print("x_kernel_shape",x.get_weights()[0].shape)
print(x.get_weights()[0])
print("y_shape",y.shape)
print(y)
"""
#-----------------
print("input")
print(inputs[0])
print("x_kernel_shape",x.get_weights()[0].shape)
print(x.get_weights()[0][:,:,0,:])
print("y_shape",y.shape)
print(y[:,:,:,0])
"""
with open("temp",'a',newline='') as aa:
    writer = csv.writer(aa, delimiter=',')
    
    writer.writerow("input")
    for i in inputs:
        for j in inputs[i]:
            writer.writerow(j)
    writer.writerow("x_kernel")
    for i in x.get_weights()[0]:
        for j in inputs[i]:
            writer.writerow(j)
    writer.writerow("nput")
    writer.writerow("input")
    writer.writerow("input")
    """

input
tf.Tensor(
[[[ 0.  1.]
  [ 2.  3.]
  [ 4.  5.]]

 [[ 6.  7.]
  [ 8.  9.]
  [10. 11.]]

 [[12. 13.]
  [14. 15.]
  [16. 17.]]

 [[18. 19.]
  [20. 21.]
  [22. 23.]]], shape=(4, 3, 2), dtype=float32)
x_kernel_shape (5, 6, 7, 2)
[[[-0.09983869  0.1196986 ]
  [ 0.00953311 -0.141326  ]
  [ 0.03965412  0.02564952]
  [-0.02767743 -0.14619054]
  [-0.13162328  0.11858912]
  [-0.12110876 -0.01289244]]

 [[ 0.0732338  -0.08781353]
  [-0.10009018 -0.10044247]
  [ 0.04564823  0.04654509]
  [-0.05508351 -0.02642491]
  [ 0.06559788 -0.13119934]
  [-0.00115722  0.13820441]]

 [[-0.01880012 -0.029322  ]
  [ 0.05729961 -0.05572915]
  [-0.1180751  -0.00366293]
  [-0.07721225  0.07153495]
  [ 0.14271335  0.13773818]
  [ 0.0422394  -0.11171082]]

 [[-0.03273585  0.00369747]
  [-0.00540996  0.01318371]
  [-0.02321938 -0.05226301]
  [ 0.09153588 -0.09140637]
  [-0.09514656  0.06691791]
  [-0.06186067  0.08842036]]

 [[-0.12492818 -0.14041954]
  [-0.04637683 -0.07089385]
  [-0.03567501 -0.06285203]
  [-0.

'\nwith open("temp",\'a\',newline=\'\') as aa:\n    writer = csv.writer(aa, delimiter=\',\')\n    \n    writer.writerow("input")\n    for i in inputs:\n        for j in inputs[i]:\n            writer.writerow(j)\n    writer.writerow("x_kernel")\n    for i in x.get_weights()[0]:\n        for j in inputs[i]:\n            writer.writerow(j)\n    writer.writerow("nput")\n    writer.writerow("input")\n    writer.writerow("input")\n    '

In [17]:
def cnn_reg_alloc_batch_normal_layer(self, input_shape, hconfig):
    pm = {}
    rescale = get_conf_param(hconfig, 'rescale', True)
    pm['epsilon'] = get_conf_param(hconfig, 'epsilon', 1e-10)
    pm['exp_ratio'] = get_conf_param(hconfig, 'exp_ratio', 0.001)
    #print("오ㅓ잉",input_shape)
    bn_dim = input_shape
    #print("악",bn_dim)
    pm['mavg'] = np.zeros(bn_dim, dtype = 'float32')
    pm['mvar'] = np.ones(bn_dim, dtype = 'float32')
    if rescale:
        pm['scale'] = np.ones(bn_dim, dtype = 'float32')
        pm['shift'] = np.zeros(bn_dim, dtype = 'float32')
    return pm, input_shape
    
def cnn_reg_forward_batch_normal_layer(self, x, hconfig, pm):
    x_shape=tuple(x.shape[1:])
    if self.is_training:
        x_flat = x.reshape([x.shape[0], -1])
        avg = np.mean(x_flat, axis=0).reshape(x_shape)
        var = np.var(x_flat, axis=0).reshape(x_shape)
        #print(avg.shape,var.shape)
        #print(pm['mavg'].shape)
        #print(pm['exp_ratio'])
        #print(avg.shape)
        #print(pm['mavg'].shape)
        pm['mavg'] += pm['exp_ratio'] * (avg - pm['mavg'])
        pm['mvar'] += pm['exp_ratio'] * (var - pm['mvar'])
    else:
        avg = pm['mavg']
        var = pm['mvar']
    std = np.sqrt(var + pm['epsilon'])
    #print(type(std))
    y = norm_x = (x - avg) / std
    if 'scale' in pm:
        y = pm['scale'] * norm_x + pm['shift']
    return y, [norm_x, std]

def cnn_reg_backprop_batch_normal_layer(self, G_y, hconfig, pm, aux):
    #print("종우")
    norm_x, std = aux
    #print(G_y.shape)
    if 'scale' in pm:
        if len(G_y.shape) == 2: axis = 0
        else: axis = 0
        G_scale = np.sum(G_y*norm_x, axis=axis)
        G_shift = np.sum(G_y, axis=axis)
        G_y = G_y * pm['scale']
        pm['scale'] -= self.learning_rate * G_scale
        pm['shift'] -= self.learning_rate * G_shift
    
    G_input = G_y / std
    return G_input
    
CnnRegModel.alloc_batch_normal_layer = cnn_reg_alloc_batch_normal_layer
CnnRegModel.forward_batch_normal_layer = cnn_reg_forward_batch_normal_layer
CnnRegModel.backprop_batch_normal_layer = cnn_reg_backprop_batch_normal_layer


[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
[[ 6  7  8]
 [ 9 10 11]]
(2, 3, 4)
(2, 3)
[10. 11. 12. 13.]


In [5]:
import numpy as np
a=list([[1,2,3],[4,5,6]])

print(a[1][2])

6


In [3]:
a=({"te":5})
print(a["te"])

5


In [12]:
import cupy as np

a=np.array([[[1,2,3],[4, 5, 6]],[[7,8,9],[10,11,12]]])
b=np.array([[1,2],[3,4]])
#a_=np.sum(a,axis=(1,2))
print(a)
print(a[0].transpose())
print(a.transpose()[0])
#print(np.matmul(a,b))
#print(np.matmul(np.reshape(a,(-1,a.shape[-1])),b))

[[[ 1  2  3]
  [ 4  5  6]]

 [[ 7  8  9]
  [10 11 12]]]
[[1 4]
 [2 5]
 [3 6]]
[[ 1  7]
 [ 4 10]]
